In [1]:
import pandas as pd
from Bio.Seq import Seq
from veliadb.base import Session, Assembly

In [2]:
metaorfs = pd.read_table('s3://velia-piperuns-dev/VPR_orfcalling_20240307222241_iPSC-rep2_SRR9113065/output/VPR_orfcalling_20240307222241_iPSC-rep2_SRR9113065_orf_features.csv')

In [3]:
session = Session()
ucsc_style2assembly_id = {}
for a in session.query(Assembly).all():
    if a.ucsc_style_name != 'na':
        if "_" in a.ucsc_style_name:
            chrom_name = a.genbank_accession
        else:
            chrom_name = a.ucsc_style_name        
        ucsc_style2assembly_id[chrom_name] = a.id

In [4]:
def exon_block_to_veliadb_style(exon_blocks, exon_delimiter = '|'):
    block_sizes = []
    chrom_starts = []
    for e in exon_blocks.split(exon_delimiter):
        e_start, e_end = e.split('-')
        block_sizes.append(str(int(e_end)-int(e_start)))
        chrom_starts.append(str(int(e_start)+1)) # Add 1 to starts to convert to 1-indexed
    return ';'.join(chrom_starts), ';'.join(block_sizes)

def metaorf_call_to_veliadb_hash_string(row, chrom_id_to_assembly_id, bp_df_ids):
    assembly_id = chrom_id_to_assembly_id[row['chrom_id']]
    chrom_starts, block_sizes = exon_block_to_veliadb_style(row['exon_blocks'])
    chrom_start = int(row['orf_start'])+1 # Add 1 to start to convert to 1-indexed
    hash_string = f"{assembly_id}_{chrom_start}_{row['orf_end']}_{row['strand']}_{chrom_starts}_{block_sizes}"
    
    if hash_string in bp_df_ids:
        return hash_string
    else:
        return ""

In [5]:
bp_df = pd.read_csv('s3://velia-data-dev/VDC_004_annotation/big_prot/v0.8.1_minlen_15_maxlen_150/orfset_v0.8.1_minlen_15_maxlen_150_orfs.csv.gz')

KeyboardInterrupt: 

In [ ]:
bp_df_ids = set(bp_df["orf.orf_idx_str"])

In [ ]:
metaorfs["bigprot_id"] = metaorfs.apply(lambda x: metaorf_call_to_veliadb_hash_string(x, ucsc_style2assembly_id, bp_df_ids), axis=1)

In [ ]:
metaorfs["bigprot_id"] == ""

In [ ]:
metaorfs.index[0]

In [ ]:
metaorfs

In [ ]:
bp_df = pd.read_csv('s3://velia-data-dev/VDC_004_annotation/big_prot/v0.8.1_minlen_15_maxlen_150/orfset_v0.8.1_minlen_15_maxlen_150_orfs.csv.gz')

In [ ]:
bp_df = bp_df.set_index("orf.orf_idx_str")

In [ ]:
bp_df

In [ ]:
count = 0
for i in range(len(metaorfs.index)):
    if (metaorfs.index[i] not in bp_df.index
        and metaorfs.iloc[i]["orf_sequence"][:3] in ["ATG", "CTG", "GTG", "TTG"]
        and len(metaorfs.iloc[i]["orf_sequence"]) >= 15
        and len(metaorfs.iloc[i]["orf_sequence"]) <= 150):
        if "+" in metaorfs.index[i]:
            count += 1
            #if count < 10 and "-" in metaorfs.index[i]:
        print(i, metaorfs.index[i], metaorfs.iloc[i]["orf_sequence"][:3])
        break
print(count)

In [ ]:
metaorfs.index

In [ ]:
len(metaorfs.index)

In [ ]:
for row in bp_df.index:
    if row.startswith("1_120952639"):
        print(row)

In [ ]:
metaorfs.index[0]

In [ ]:
metaorfs.loc["1_120952639_120952677_+_120952639_39"]

In [ ]:
bp_df.loc[metaorfs.index[1]]